In [3]:
# %load std_libraries.py
#to keep my scripts consistent, I am adding this to the top of all scripts with %load std_libraries.py
import sys
#Indicate operating environment and import core modules
location_input = input("what computer are you on? a = Bens, b = gpucluster, c = other   ")
location_dict = {'a': "C:\\Users\\heine\\github\\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data",'c':'you need to add your location to the location_dict'}
base_dir = location_dict[location_input]
print("base directory is " + base_dir)

if sys.path[-1] != base_dir:
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path: " )
    print(sys.path)

import os

print("I am about to import a library")
from core import expression_plots 
from core import io_library 
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook
data_processing_dir = base_dir + os.sep + os.path.normpath("expression_data") + os.sep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#from matplotlib_venn import venn2
#import seaborn as sns; sns.set(style="ticks", color_codes=True)
#from sklearn import linear_model
import pickle
#import subprocess
#import networkx as nx
#import scipy.stats as stats
#import statsmodels.graphics.gofplots as stats_graph
#import scipy.cluster.hierarchy as sch

# from Bio import SeqIO
# from Bio import SeqFeature as sf
# from Bio.Alphabet import generic_dna
# from Bio.Seq import Seq

# import re

from collections import Counter
# import scipy.stats as stats
# from itertools import chain
#from itertools import product
#this only works if you are online
online_input = input("are you online? Yes/No")
if online_input == "Yes": 
    import plotly.plotly as py
    import plotly.graph_objs as pygo
    import plotly.tools as pytools
    py.sign_in('heineike02_student','9dMTMZgJMgUP0YX0P5mQ')
    #py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')
    
# for phylogenetic trees: 
# from ete3 import Tree

#for scraping internet data (e.g. ncbi)
#import requests
#from lxml import etree    #parses xml output

what computer are you on? a = Bens, b = gpucluster, c = other   a
base directory is C:\Users\heine\github\expression_broad_data
I am about to import a library
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
are you online? Yes/NoNo


In [4]:
# Import KL/SC data
fname_cross_spec_comparison = data_processing_dir + os.path.normpath("20170817_klscpka/cross_species_compare_ASminus_ASplus_YPD.csv")
kl_sc_PKA_data = pd.read_csv(fname_cross_spec_comparison, index_col=0)

#Load group lists from promoter analysis
fname = os.path.normpath(data_processing_dir + '20170817_klscpka/20180410_act_sets_P1.pkl')
with open(fname, mode = 'rb') as f: 
    gene_list_dict = pickle.load(f)
    



In [5]:
gene_list_dict.keys()
 total = len(gene_list_dict[gene_set_name])

dict_keys(['sc_act', 'kl_act', 'sc_only_act', 'kl_only_act', 'sc_sckl_act', 'kl_sckl_act', 'sc_klonly_act', 'kl_sconly_act'])

In [6]:
for key, value in gene_list_dict.items():
    print(key)
    print(len(value))


sc_act
184
kl_act
71
sc_only_act
156
kl_only_act
47
sc_sckl_act
28
kl_sckl_act
24
sc_klonly_act
27
kl_sconly_act
112


In [19]:
#Find paralogs and add paralog groups
pka_subsets = ['sc_only_act', 'sc_sckl_act', 'sc_klonly_act']

gene_sets = {}
for pka_subset in pka_subsets: 

    gene_sets_subset = {}

    kl_sc_PKA_data_subset_paralogs = kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(gene_list_dict[pka_subset]) & (kl_sc_PKA_data['N SCer Orthologs']==2)]
    kl_genename_counts = Counter(kl_sc_PKA_data_subset_paralogs['kl_genename'])

    paralogs_diff_ind_act_kl_genename = [kl_genename for kl_genename, NN in kl_genename_counts.items() if NN==1]
    #get the active paralog of the differentially induced paralogs
    paralogs_diff_ind_act = list(kl_sc_PKA_data_subset_paralogs[kl_sc_PKA_data_subset_paralogs['kl_genename'].isin(paralogs_diff_ind_act_kl_genename)]['sc_genename'])
    gene_sets_subset['paralogs_diff_ind_act']= paralogs_diff_ind_act

    paralogs_diff_ind_low = list(io_library.get_other_paralogs_from_dataframe(io_library.SC_common_name_lookup(paralogs_diff_ind_act), kl_sc_PKA_data)['sc_genename'])
    gene_sets_subset['paralogs_diff_ind_low'] = paralogs_diff_ind_low

    paralogs_similar_ind_kl_genename = [kl_genename for kl_genename, NN in kl_genename_counts.items() if NN==2]
    #all of these should be in the subset already
    paralogs_similar_ind = list(kl_sc_PKA_data_subset_paralogs[kl_sc_PKA_data_subset_paralogs['kl_genename'].isin(paralogs_similar_ind_kl_genename)]['sc_genename'])
    gene_sets_subset['paralogs_similar_ind'] = paralogs_similar_ind                            

    kl_sc_PKA_data_subset_no_paralogs = kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(gene_list_dict[pka_subset]) & (kl_sc_PKA_data['N SCer Orthologs']==1)]
    no_paralog = list(kl_sc_PKA_data_subset_no_paralogs['sc_genename'])
    gene_sets_subset['no_paralog'] = no_paralog

    gene_sets[pka_subset] = gene_sets_subset

gene_sets

{'sc_klonly_act': {'no_paralog': ['YJL057C',
   'YHR044C',
   'YBL016W',
   'YPR026W',
   'YGL036W',
   'YBR057C',
   'YPL054W',
   'YBR149W',
   'YLL019C',
   'YML070W',
   'YIL024C',
   'YMR063W',
   'YGL081W',
   'YDR446W',
   'YPR002W',
   'YOR386W',
   'YOL082W',
   'YNL010W',
   'YHR005C',
   'YIR017C',
   'YAL008W',
   'YDR227W',
   'YDR051C'],
  'paralogs_diff_ind_act': [],
  'paralogs_diff_ind_low': [],
  'paralogs_similar_ind': ['YEL060C', 'YOR003W', 'YDR259C', 'YOR028C']},
 'sc_only_act': {'no_paralog': ['YHR087W',
   'YML128C',
   'YPL222W',
   'YIL155C',
   'YHL021C',
   'YML004C',
   'YOR161C',
   'YLR149C',
   'YKL086W',
   'YKL065W-A',
   'YBR126C',
   'YOR374W',
   'YLR251W',
   'YMR196W',
   'YOR348C',
   'YOL084W',
   'YFL014W',
   'YBL064C',
   'YGL010W',
   'YKL151C',
   'YPR151C',
   'YHR140W',
   'YDR119W-A',
   'YKL103C',
   'YHR104W',
   'YHL024W',
   'YNL274C',
   'YBR230C',
   'YBL078C',
   'YNL200C',
   'YGR088W',
   'YLL039C',
   'YDR171W',
   'YPL123C',
  

In [50]:
gene_sets_table = pd.DataFrame(index = ['Total', 'Total w/ Ortholog', 
                                        'Diff Induced Paralogs (high induction)', 'Diff Induced Paralogs (low induction)',
                                        'Similarly Induced Paralogs', 'No Paralogs'])
for gene_set_name, gene_set in gene_sets.items():
    table_data = []
    total = len(gene_list_dict[gene_set_name])
    table_data.append(total)
    print(gene_set_name + " Total: " + str(total))
    
    total_orth = len(kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(gene_list_dict[gene_set_name])])
    table_data.append(total_orth)
    print("Total with ortholog: " + str(total_orth))
    for gene_subset_name, gene_subset in gene_set.items():
        table_data.append(len(gene_subset))
        print(gene_subset_name + ": " + str(len(gene_subset)))
    gene_sets_table[gene_set_name]=table_data
gene_sets_table.rename(columns={'sc_only_act': 'SCer Only Activated', 'sc_sckl_act': 'SCer and KLac Activated', 'sc_klonly_act': 'KLac Only Activated'})

sc_only_act Total: 156
Total with ortholog: 112
paralogs_diff_ind_act: 53
paralogs_diff_ind_low: 53
paralogs_similar_ind: 4
no_paralog: 55
sc_sckl_act Total: 28
Total with ortholog: 28
paralogs_diff_ind_act: 0
paralogs_diff_ind_low: 0
paralogs_similar_ind: 8
no_paralog: 20
sc_klonly_act Total: 27
Total with ortholog: 27
paralogs_diff_ind_act: 0
paralogs_diff_ind_low: 0
paralogs_similar_ind: 4
no_paralog: 23


,SCer Only Activated,SCer and KLac Activated,KLac Only Activated
Total,156,28,27
Total w/ Ortholog,112,28,27
Diff Induced Paralogs (high induction),53,0,0
Diff Induced Paralogs (low induction),53,0,0
Similarly Induced Paralogs,4,8,4
No Paralogs,55,20,23


In [48]:
cmap = cm.get_cmap('tab10')
color = cmap(1/10)
'rgba'+str(color)

'rgba(1.0, 0.49803921568627452, 0.054901960784313725, 1.0)'

In [49]:
#Plot raw expression for various gene sets

hover_text = []
for ind, row in kl_sc_PKA_data.iterrows():
    hover_text.append(('{SC_common_name}<br>kl cont: {klmin:.2f}<br>kl nmpp1: {klnmpp1:.2f}').format(
                              SC_common_name=row['SC_common_name'],
                              klmin = row['rlog_AS_noNMPP1_YPD_t40_mean_KL'],
                              klnmpp1 = row['rlog_AS_yesNMPP1_YPD_t40_mean_KL']
                              ))

kl_sc_PKA_data['hover'] = hover_text

cmap = cm.get_cmap('tab10')

data = []
NN = 0
for gene_set_name, gene_set in gene_sets.items():
     for gene_subset_name, gene_subset in gene_set.items():
        kl_sc_PKA_data_subset = kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(gene_subset)]

        trace = pygo.Scatter(
            x = kl_sc_PKA_data_subset['rlog_AS_noNMPP1_YPD_t40_mean_SC'], 
            y =  kl_sc_PKA_data_subset['rlog_AS_yesNMPP1_YPD_t40_mean_SC'],
            text = kl_sc_PKA_data_subset['hover'],
            mode = 'markers',
            marker = {'opacity': 0.5, 
                      'color': 'rgba'+str(cmap(NN/10))}, 
            name = gene_set_name + ': ' + gene_subset_name
        )
        
        data.append(trace)
        NN = NN+1

layout = pygo.Layout(
    xaxis= {
        #"range":[-2, 20],
        "title":'SC rlog control'
    },
    yaxis= {
        #"range":[-2, 20],
        "title":'SC rlog + NMPP1'
    }
)


fig = pygo.Figure(data=data, layout = layout)
py.iplot(fig, filename='raw_expression_PKA_activated_genes')